In [48]:
import pandas as pd

In [49]:
# Pull data from olympic dataset
# Pull in word pop and noc_regions to merge data for accurate country name, country code, and athlete participants

dataset = pd.read_csv('./athlete_events.csv')
regions = pd.read_csv('world_population.csv', usecols=['Team', 'code'])
noc_regions = pd.read_csv('noc_regions.csv', usecols=['NOC', 'region'])

dataset.head(10)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
7,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
8,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN
9,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,"Speed Skating Women's 1,000 metres",NaN


In [50]:
filtered_dataset_2016_summer_dup = dataset[(dataset['Year'] == 2016) & (dataset['Season'] == 'Summer')]
filtered_dataset_2016_summer = filtered_dataset_2016_summer.drop_duplicates(subset=['ID'])
filtered_dataset_2016_summer.head(10)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
80,22,Andreea Aanei,F,22.0,170.0,125.0,Romania,ROU,2016 Summer,2016,Summer,Rio de Janeiro,Weightlifting,Weightlifting Women's Super-Heavyweight,NaN
139,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,NaN
148,55,Antonio Abadia Beci,M,26.0,170.0,65.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 5,000 metres",NaN
158,62,Giovanni Abagnale,M,21.0,198.0,90.0,Italy,ITA,2016 Summer,2016,Summer,Rio de Janeiro,Rowing,Rowing Men's Coxless Pairs,Bronze
161,65,Patimat Abakarova,F,21.0,165.0,49.0,Azerbaijan,AZE,2016 Summer,2016,Summer,Rio de Janeiro,Taekwondo,Taekwondo Women's Flyweight,Bronze
175,73,Luc Abalo,M,31.0,182.0,86.0,France,FRA,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Men's Handball,Silver
217,105,Ilyas Abbadi,M,23.0,175.0,75.0,Algeria,ALG,2016 Summer,2016,Summer,Rio de Janeiro,Boxing,Boxing Men's Middleweight,NaN
232,110,Abubakar Abbas Abbas,M,20.0,175.0,66.0,Bahrain,BRN,2016 Summer,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's 400 metres,NaN
263,135,Lois Abbingh,F,23.0,178.0,72.0,Netherlands,NED,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
267,138,Salwan Jasim Abbood Abbood,M,24.0,180.0,104.0,Iraq,IRQ,2016 Summer,2016,Summer,Rio de Janeiro,Weightlifting,Weightlifting Men's Heavyweight,NaN


In [51]:
# Total participants in 2016 olympics

row_count_2016_summer = len(filtered_dataset_2016_summer)
print("The total count of rows in the dataset is:", row_count_2016_summer)

The total count of rows in the dataset is: 11179


In [59]:
# Sort athletes and aggregate athlete count by team

athletes_per_noc_2016_summer = filtered_dataset_2016_summer.groupby('Team').size().reset_index(name='total_athletes')
print(athletes_per_noc_2016_summer)

# Merge choropleth map regions with athletes_per_noc_2016_summer
merged_dataset = athletes_per_noc_2016_summer.merge(regions, on='Team', how='left')
print(merged_dataset)

# Drop duplicates

merged_dataset = merged_dataset.drop_duplicates(subset='Team')
mask = merged_dataset['Team'].str.contains('-\d+$')
merged_dataset = merged_dataset[~mask]

merged_dataset = merged_dataset.dropna(subset=['code'])

# Rearrange and save to CSV
merged_dataset = merged_dataset.loc[:, ['Team', 'code', 'total_athletes']]
merged_dataset = merged_dataset.rename(columns={'Team': 'name', 'NOC': 'code', 'total_athletes': 'pop'})

merged_dataset.to_csv('merged_dataset.csv', index=False)

               Team  total_athletes
0       Afghanistan               3
1           Albania               6
2           Algeria              64
3    American Samoa               4
4           Andorra               4
..              ...             ...
241       Venezuela              85
242         Vietnam              22
243           Yemen               3
244          Zambia               7
245        Zimbabwe              30

[246 rows x 2 columns]
               Team  total_athletes code
0       Afghanistan               3  AFG
1           Albania               6  ALB
2           Algeria              64  DZA
3    American Samoa               4  ASM
4           Andorra               4  AND
..              ...             ...  ...
241       Venezuela              85  VEN
242         Vietnam              22  NaN
243           Yemen               3  YEM
244          Zambia               7  ZMB
245        Zimbabwe              30  ZWE

[246 rows x 3 columns]
